In [18]:
import pandas as pd
import matplotlib.pyplot as plt

import sqlalchemy
from sqlalchemy import create_engine, func, inspect, text

In [19]:
df = pd.read_csv("merged_df.csv")
df.head()

,latitude,longitude,Country,Website,Studio,Publisher,PC,Mobile,VR,VR only,Console,Browser games
0,61.924110,25.748151,Finland,10tons.com/,10tons,?,X,X,?,?,?,?
1,51.919438,19.145136,Poland,11bitstudios.com/,11 bit studios,11 bit studios,X,X,?,?,X,?
2,64.963051,-19.020835,Iceland,1939games.com/,1939 Games,?,X,?,?,?,?,?
3,46.227638,2.213749,France,1button.co/,1Button,?,?,X,?,?,?,?
4,61.524010,105.318756,Russia,1c.games/ru/,1C Games,1C,X,?,?,?,X,?


In [20]:
engine = create_engine("sqlite:///merged_df.sqlite")

df.to_sql("merged_df", con=engine, if_exists="replace", index=False)

600

In [21]:
# explore and understand the data

# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
tables = inspector.get_table_names()

# Using the inspector to print the column names within each table and its types
for table in tables:
    print(table)
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])
        
    print()

merged_df
latitude FLOAT
longitude FLOAT
Country TEXT
Website TEXT
Studio TEXT
Publisher TEXT
PC TEXT
Mobile TEXT
VR TEXT
VR only TEXT
Console TEXT
Browser games TEXT



In [22]:
df['Country'].nunique()

38

In [23]:
Country = "Russia"

# allow the user to select ALL or a specific state
if Country == "All":
    where_clause = "1=1"
else:
    where_clause = f"Country = '{Country}'"

query = f"""
        SELECT
            *
        FROM
            merged_df
        WHERE
            {where_clause};
"""

print(query)


        SELECT
            *
        FROM
            merged_df
        WHERE
            Country = 'Russia';



In [24]:
df_map = pd.read_sql(text(query), con=engine)
df_map.head()

,latitude,longitude,Country,Website,Studio,Publisher,PC,Mobile,VR,VR only,Console,Browser games
0,61.52401,105.318756,Russia,1c.games/ru/,1C Games,1C,X,?,?,?,X,?
1,61.52401,105.318756,Russia,2zombie.com/,2 Zombie Games,?,X,X,?,?,X,?
2,61.52401,105.318756,Russia,absolutsoft.com/,AbsolutSoft,?,X,?,?,?,?,?
3,61.52401,105.318756,Russia,adrovgames.com/,AdroVGames,?,X,?,?,?,?,?
4,61.52401,105.318756,Russia,aksweb.ru/,AK Studio,?,X,X,?,?,?,?


In [25]:
Country = "Finland"

# allow the user to select ALL or a specific state
if Country == "All":
    where_clause = "1=1"
else:
    where_clause = f"Country = '{Country}'"

query = f"""
        SELECT
            *
        FROM
            merged_df
        WHERE
            {where_clause};
"""

print(query)


        SELECT
            *
        FROM
            merged_df
        WHERE
            Country = 'Finland';



In [26]:
df_map = pd.read_sql(text(query), con=engine)
df_map.head()

,latitude,longitude,Country,Website,Studio,Publisher,PC,Mobile,VR,VR only,Console,Browser games
0,61.92411,25.748151,Finland,10tons.com/,10tons,?,X,X,?,?,?,?
1,61.92411,25.748151,Finland,3rdeyestudios.fi/#home-section,3rd eye studios,?,X,?,X,?,?,?
2,61.92411,25.748151,Finland,actionreaction.games/,Action reaction games,?,?,X,?,?,?,?
3,61.92411,25.748151,Finland,actionsquadstudios.com/,Action Squad Studios,Daedalic Entertainment,X,?,?,?,?,?
4,61.92411,25.748151,Finland,actnormalgames.com/,Act Normal Games,?,X,?,?,?,?,?


In [27]:
df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   latitude       20 non-null     float64
 1   longitude      20 non-null     float64
 2   Country        20 non-null     object 
 3   Website        20 non-null     object 
 4   Studio         20 non-null     object 
 5   Publisher      20 non-null     object 
 6   PC             20 non-null     object 
 7   Mobile         20 non-null     object 
 8   VR             20 non-null     object 
 9   VR only        20 non-null     object 
 10  Console        20 non-null     object 
 11  Browser games  20 non-null     object 
dtypes: float64(2), object(10)
memory usage: 2.0+ KB


In [28]:
Country = "All"

# allow the user to select ALL or a specific state
if Country == "All":
    where_clause = "1=1"
else:
    where_clause = f"Country = '{Country}'"
    
query = f"""
    SELECT
        Studio,
        Country,
        count(*) as num_studios
    FROM
        merged_df
    WHERE
        {where_clause}
    GROUP BY
        Country
    ORDER BY
        num_studios desc;
"""

df_bar = pd.read_sql(text(query), con=engine)
df_bar.head()

,Studio,Country,num_studios
0,1CC Games,UK,115
1,1C Games,Russia,67
2,11 bit studios,Poland,54
3,Alpha Blend Interactive,Germany,53
4,1Button,France,45


In [30]:
Country = "All"

# allow the user to select ALL or a specific Country
if Country == "All":
    where_clause = "1=1"
else:
    where_clause = f"Country = '{Country}'"
    
query = f"""
    SELECT
        Country,
        count(*) as num_studios
    FROM
        merged_df
    WHERE
        {where_clause}
    GROUP BY
        Country
    ORDER BY
        num_studios desc
    LIMIT 10;
"""

df_bar2 = pd.read_sql(text(query), con=engine)
df_bar2.head()

,Country,num_studios
0,UK,115
1,Russia,67
2,Poland,54
3,Germany,53
4,France,45


In [31]:
Country = "All"

# allow the user to select ALL or a specific Country
if Country == "All":
    where_clause = "1=1"
else:
    where_clause = f"Country = '{Country}'"
    
query = f"""
    SELECT
        Country,
        count(*) as num_studios
    FROM
        merged_df
    WHERE
        {where_clause}
    GROUP BY
        Country
    ORDER BY
        num_studios desc
    LIMIT 10;
"""

df_bar2 = pd.read_sql(text(query), con=engine)
df_bar2.head()

,Country,num_studios
0,UK,115
1,Russia,67
2,Poland,54
3,Germany,53
4,France,45
